In [1]:
import os, re, time, uuid, json, random, requests, psycopg2
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# ── Base de données ──────────────────────────────────────────
DB_CONFIG = {
    "host":     "localhost",
    "database": "estate_mind_db",
    "user":     "postgres",
    "password": "admin"
}

# ── Dossier média ────────────────────────────────────────────
MEDIA_ROOT = "media"
os.makedirs(MEDIA_ROOT, exist_ok=True)

# ── Paramètres de scraping ───────────────────────────────────
TARGET_PER_CATEGORY    = 300   # 300 vente + 300 location = 600 par type
MAX_PAGES_PER_CATEGORY = 15    # ~25 annonces/page → 15 pages suffisent

# ── Catégories (slug → transaction_type, property_type) ─────
CATEGORIES = {
    "sc/appartements-a-vendre":          ("sale", "apartment"),
    "sc/appartements-a-louer":           ("rent", "apartment"),
    "sc/villas-a-vendre":                ("sale", "villa"),
    "sc/villas-a-louer":                 ("rent", "villa"),
    "sc/terrains-a-vendre":              ("sale", "land"),
    "sc/terrains-a-louer":              ("rent", "land"),
    "sc/bureaux-et-plateaux-a-vendre":   ("sale", "office"),
    "sc/bureaux-et-plateaux-a-louer":    ("rent", "office"),
    "sc/maisons-a-vendre":               ("sale", "house"),
    "sc/maisons-a-louer":                ("rent", "house"),
}

# ── Mapping features FR → EN ─────────────────────────────────
FEATURE_MAPPING = {
    "Climatisation":     "has_air_conditioning",
    "Chauffage central": "has_central_heating",
    "Ascenseur":         "has_elevator",
    "Jardin":            "has_garden",
    "Piscine":           "has_pool",
    "Terrasse":          "has_terrace",
    "Garage":            "has_garage",
    "Parking":           "has_parking",
    "Vue sur mer":       "has_sea_view",
    "Meublé":            "is_furnished",
    "Cuisine équipée":   "has_equipped_kitchen",
    "Interphone":        "has_intercom",
    "Balcon":            "has_balcony",
    "Cave":              "has_cellar",
    "Sécurité":          "has_security",
    "Digicode":          "has_digicode",
    "Vidéosurveillance": "has_cctv",
}

print("✅ Configuration chargée.")

✅ Configuration chargée.


In [2]:
# ════════════════════════════════════════════════════════════
# 1. CRÉATION TABLE (depuis zéro)
# ════════════════════════════════════════════════════════════

conn = psycopg2.connect(**DB_CONFIG)
cur  = conn.cursor()

cur.execute("DROP TABLE IF EXISTS mubawab_listings;")
cur.execute("""
    CREATE TABLE mubawab_listings (
        id               SERIAL PRIMARY KEY,

        -- Infos principales
        title            TEXT,
        subtitle         TEXT,
        price            VARCHAR(100),
        transaction_type VARCHAR(20),    -- 'sale' | 'rent'
        property_type    VARCHAR(30),    -- 'apartment' | 'villa' | 'land' | 'office' | 'house'

        -- Géographie structurée
        location_details JSONB,          -- {city, municipality, district, zone, raw}
        municipality     VARCHAR(100),   -- extrait de location_details pour requêtes rapides

        -- Caractéristiques numériques (extraites intelligemment du titre/description)
        surface          FLOAT,          -- superficie en m² (ex: 132.0)
        rooms            INTEGER,        -- nombre de pièces (ex: 2 pour S2)
        bedrooms         VARCHAR(30),    -- brut depuis la page détail
        bathrooms        VARCHAR(30),    -- brut depuis la page détail

        -- Données riches
        features         JSONB,          -- {has_elevator: true, has_garden: true, ...}
        poi              JSONB,          -- points d'intérêt proches

        -- Texte
        description      TEXT,

        -- Source
        url              TEXT UNIQUE,

        -- Images
        image_path       TEXT,           -- chemin miniature principale
        images_folder    TEXT,           -- dossier contenant toutes les images
        images_count     INTEGER DEFAULT 0,

        -- Métadonnées
        scraped_at       TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        last_updated     VARCHAR(100)    -- date affichée sur l'annonce
    );
""")
conn.commit()
cur.close()
conn.close()
print("✅ Table 'mubawab_listings' créée avec succès.")


# ════════════════════════════════════════════════════════════
# 2. FONCTIONS UTILITAIRES
# ════════════════════════════════════════════════════════════

def get_geo_zone(city: str) -> str:
    c = city.lower()
    if any(x in c for x in ["tunis","ariana","ben arous","manouba","marsa",
                              "carthage","la goulette","bardo","ezzahra"]):
        return "grand_tunis"
    if any(x in c for x in ["sousse","monastir","mahdia"]):
        return "sahel"
    if any(x in c for x in ["sfax","gabes","jerba","djerba","medenine",
                              "tataouine","tozeur","kebili","zarzis"]):
        return "sud"
    if any(x in c for x in ["bizerte","nabeul","hammamet","beja",
                              "jendouba","zaghouan","kelibia","korba"]):
        return "nord"
    if any(x in c for x in ["kairouan","kasserine","sidi bouzid","siliana","gafsa"]):
        return "centre"
    return "autre"

def build_location(subtitle: str) -> tuple[dict, str]:
    """Retourne (dict JSON, municipality string)"""
    parts = [p.strip() for p in subtitle.split(',')]
    city         = parts[-1] if parts else "Tunisie"
    municipality = parts[-2] if len(parts) >= 2 else city
    district     = parts[0]  if len(parts) >= 3 else ""
    loc = {
        "city":         city,
        "municipality": municipality,
        "district":     district,
        "zone":         get_geo_zone(city),
        "raw":          subtitle
    }
    return loc, municipality

def extract_rooms(text: str) -> int | None:
    """S2, s+2, S 2, F3, T4, studio, 2 pièces, 2 chambres → entier"""
    if not text:
        return None
    t = text.lower()
    for pat in [
        r'\bs\s*\+?\s*(\d)\b',        # S2, S+2, s 2
        r'\b[ft]\s*(\d)\b',            # F3, T4
        r'(\d)\s*pi[eè]ces?',          # 3 pièces
        r'(\d)\s*chambres?',           # 2 chambres
        r'\bstudio\b',                 # studio = 1
    ]:
        m = re.search(pat, t)
        if m:
            return 1 if 'studio' in pat else (int(m.group(1)) if 1 <= int(m.group(1)) <= 10 else None)
    return None

def extract_surface(text: str) -> float | None:
    """45m², 132 m2, superficie de 85 → float"""
    if not text:
        return None
    t = text.lower()
    for pat in [
        r'(\d[\d\s.,]*)\s*m\s*[²2]',
        r'(?:superficie|surface|superfici)\s*(?:de|:)?\s*(\d[\d.,]*)',
        r'de\s+(\d[\d.,]*)\s*m\s*[²2]',
    ]:
        m = re.search(pat, t)
        if m:
            try:
                val = float(m.group(1).replace(' ','').replace(',','.'))
                if 10 <= val <= 10000:
                    return val
            except:
                continue
    return None

def smart_extract(title: str, description: str) -> tuple[int | None, float | None]:
    """Retourne (rooms, surface) en cherchant dans titre puis description."""
    rooms   = extract_rooms(title)   or extract_rooms(description)
    surface = extract_surface(title) or extract_surface(description)
    return rooms, surface

def map_features(raw_list: list) -> dict:
    result = {}
    for item in raw_list:
        item = item.strip()
        if not item:
            continue
        mapped = False
        for fr_key, en_key in FEATURE_MAPPING.items():
            if fr_key.lower() in item.lower():
                result[en_key] = True
                mapped = True
                break
        if not mapped and len(item) < 60:
            result[item] = True
    return result

def download_images(img_urls: list, prop_type: str, listing_id: int) -> tuple[str | None, str, int]:
    """
    Télécharge toutes les images dans media/{prop_type}/{listing_id}/
    Retourne (chemin_miniature, chemin_dossier, nb_images)
    """
    if not img_urls:
        folder = os.path.join(MEDIA_ROOT, prop_type, str(listing_id))
        return None, folder, 0

    folder = os.path.join(MEDIA_ROOT, prop_type, str(listing_id))
    os.makedirs(folder, exist_ok=True)

    saved = []
    for idx, url in enumerate(img_urls, start=1):
        if not url or url.startswith("data:"):
            continue
        try:
            r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=8)
            if r.status_code == 200:
                ext = url.split(".")[-1].split("?")[0]
                ext = ext if ext in ["jpg","jpeg","png","webp"] else "jpg"
                filepath = os.path.join(folder, f"image_{idx:03d}.{ext}")
                with open(filepath, "wb") as f:
                    f.write(r.content)
                saved.append(filepath)
                time.sleep(0.15)
        except:
            continue

    thumb = saved[0] if saved else None
    return thumb, folder, len(saved)


# ════════════════════════════════════════════════════════════
# 3. SELENIUM
# ════════════════════════════════════════════════════════════

def create_driver():
    opts = Options()
    opts.add_argument("--headless=new")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument("--window-size=1920,1080")
    opts.add_argument("--disable-blink-features=AutomationControlled")
    opts.add_experimental_option("excludeSwitches", ["enable-automation"])
    opts.add_experimental_option("useAutomationExtension", False)
    opts.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    )
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()), options=opts
    )
    driver.execute_script(
        "Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"
    )
    return driver


# ════════════════════════════════════════════════════════════
# 4. SCRAPING PAGE DÉTAIL
# ════════════════════════════════════════════════════════════

def scrape_detail(driver, url: str) -> dict:
    try:
        driver.get(url)
        time.sleep(random.uniform(2.5, 4))
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight / 2);")
        time.sleep(0.8)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1.5)

        soup = BeautifulSoup(driver.page_source, "html.parser")
        data = {}

        # Description
        for sel in [".blockProp p", ".description p", "#adDescription", ".adDesc"]:
            el = soup.select_one(sel)
            if el:
                data["description"] = el.get_text(" ", strip=True)
                break
        data.setdefault("description", "")

        # Date
        for sel in [".creationDate", "#creationDate", ".adDate", ".listingDate"]:
            el = soup.select_one(sel)
            if el:
                data["last_updated"] = el.get_text(strip=True)
                break
        data.setdefault("last_updated", "Unknown")

        # Chambres / salles de bain (brut, page détail)
        for feat in soup.select(".adMainFeature, .adFeature, .listingFeature"):
            txt = feat.get_text(strip=True)
            if "Chambre" in txt:
                data["bedrooms"] = txt.replace("Chambres","").replace("Chambre","").strip()
            elif "Bain" in txt or "Salle" in txt:
                data["bathrooms"] = txt.replace("Salles de bains","").replace("Salle de bain","").strip()

        # Features
        raw_feats = [li.get_text(strip=True) for li in soup.select(
            ".adDetailsList li, .features li, .equipements li, ul.featureList li"
        )]
        data["features"] = map_features(raw_feats)

        # POI
        poi = {}
        for p in soup.select(".poiList li, .poi li, .nearbyList li"):
            txt = p.get_text(strip=True)
            if txt:
                poi[txt] = "detected"
        data["poi"] = poi

        # Images (toutes)
        img_urls = []
        seen     = set()
        for sel in [".sliderPhoto img", ".gallery img", ".photoGallery img",
                    "#listingGallery img", ".adGallery img", "img.bigPhoto"]:
            for img in soup.select(sel):
                src = img.get("data-big") or img.get("data-src") or img.get("src","")
                if src and "http" in src and src not in seen:
                    img_urls.append(re.sub(r'_thumb|_small|_medium','', src))
                    seen.add(src)

        # Fallback
        if not img_urls:
            for img in soup.select("img"):
                src = img.get("data-src") or img.get("src","")
                if (src and "http" in src and src not in seen
                        and any(e in src.lower() for e in [".jpg",".jpeg",".png",".webp"])
                        and "logo" not in src.lower() and "icon" not in src.lower()):
                    img_urls.append(src)
                    seen.add(src)

        data["img_urls"] = img_urls
        return data

    except Exception as e:
        print(f"      ⚠️ Détail error: {e}")
        return {}


# ════════════════════════════════════════════════════════════
# 5. SCRAPING PAGE LISTE
# ════════════════════════════════════════════════════════════

def scrape_list_page(driver, url: str):
    driver.get(url)
    time.sleep(random.uniform(3, 5))
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight/2);")
    time.sleep(1)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    for sel in ["li.listingBox", "li.promotionListing", ".ulListing li",
                "div.listingBox", "article.listing", "[data-url]"]:
        found = soup.select(sel)
        if found:
            return found, sel
    return [], None

def extract_basic(item, trans_type, prop_type):
    try:
        link = (
            item.select_one("h2.listingTit a") or
            item.select_one("h2 a") or
            item.select_one("a[href*='/fr/']")
        )
        if not link:
            return None

        title = link.get_text(strip=True)
        url   = link.get("href","")
        if url and not url.startswith("http"):
            url = "https://www.mubawab.tn" + url
        if not url:
            return None

        price_el = item.select_one("span.priceTag, .price, [class*='price']")
        price = price_el.get_text(strip=True).replace("\xa0","").replace("\u202f","") \
                if price_el else "N/A"

        sub_el   = item.select_one("span.listingDetails, .adLocation, .location")
        subtitle = sub_el.get_text(strip=True) if sub_el else "Tunisie"

        img_el   = (item.select_one("img.photo-listing") or
                    item.select_one("img.listingImg") or
                    item.select_one(".imgContainer img") or
                    item.select_one("img"))
        thumb = None
        if img_el:
            thumb = img_el.get("data-src") or img_el.get("data-lazy") or img_el.get("src")
            if thumb and thumb.startswith("//"):
                thumb = "https:" + thumb

        return {
            "title": title, "subtitle": subtitle, "price": price,
            "transaction_type": trans_type, "property_type": prop_type,
            "thumb": thumb, "url": url,
        }
    except:
        return None


print("✅ Table créée + toutes les fonctions prêtes.")

✅ Table 'mubawab_listings' créée avec succès.
✅ Table créée + toutes les fonctions prêtes.


In [ ]:
conn = psycopg2.connect(**DB_CONFIG)
cur  = conn.cursor()
driver = create_driver()

total_saved    = 0
category_stats = {}

print("🚀 Démarrage — Objectif : 3000 annonces\n")
print(f"   Structure images : {MEDIA_ROOT}/{{property_type}}/{{id}}/image_001.jpg\n")

try:
    for slug, (trans_type, prop_type) in CATEGORIES.items():
        cat_key = f"{prop_type}_{trans_type}"
        cat_saved = 0

        print(f"\n{'═'*60}")
        print(f"  📦  {prop_type.upper()}  |  {trans_type.upper()}")
        print(f"{'═'*60}")

        for page_num in range(1, MAX_PAGES_PER_CATEGORY + 1):

            if cat_saved >= TARGET_PER_CATEGORY:
                print(f"  🎯 Objectif {TARGET_PER_CATEGORY} atteint.")
                break

            url_page = (
                f"https://www.mubawab.tn/fr/{slug}"
                if page_num == 1
                else f"https://www.mubawab.tn/fr/{slug}:p:{page_num}"
            )
            print(f"\n  📄 Page {page_num} → {url_page}")

            listings_raw, used_sel = scrape_list_page(driver, url_page)

            if not listings_raw:
                print("  🛑 Aucune annonce. Fin de catégorie.")
                break

            print(f"      → {len(listings_raw)} annonces [sel: '{used_sel}']")

            for item in listings_raw:

                if cat_saved >= TARGET_PER_CATEGORY:
                    break

                basic = extract_basic(item, trans_type, prop_type)
                if not basic:
                    continue

                # Anti-doublon
                cur.execute("SELECT id FROM mubawab_listings WHERE url=%s", (basic["url"],))
                if cur.fetchone():
                    continue

                print(f"      ⬇️  [{cat_saved+1}/{TARGET_PER_CATEGORY}] "
                      f"{basic['title'][:48]}...")

                # ── Page détail ───────────────────────────────
                det = scrape_detail(driver, basic["url"])

                title       = basic["title"]
                description = det.get("description", "")

                # ── Extraction intelligente rooms & surface ───
                rooms, surface = smart_extract(title, description)

                # Fallback : si pas trouvé dans texte, essayer bedrooms brut
                if not rooms and det.get("bedrooms"):
                    rooms = extract_rooms(det["bedrooms"])

                # ── Géographie ────────────────────────────────
                loc_dict, municipality = build_location(basic["subtitle"])

                # ── Insertion (RETURNING id) ──────────────────
                try:
                    cur.execute("""
                        INSERT INTO mubawab_listings
                            (title, subtitle, price,
                             transaction_type, property_type,
                             location_details, municipality,
                             surface, rooms,
                             bedrooms, bathrooms,
                             features, poi, description, url,
                             image_path, images_folder, images_count,
                             last_updated)
                        VALUES
                            (%s,%s,%s, %s,%s, %s,%s, %s,%s, %s,%s,
                             %s,%s,%s,%s, %s,%s,%s, %s)
                        ON CONFLICT (url) DO NOTHING
                        RETURNING id
                    """, (
                        title,
                        basic["subtitle"],
                        basic["price"],
                        trans_type,
                        prop_type,
                        json.dumps(loc_dict),
                        municipality,
                        surface,                           # FLOAT ou None
                        rooms,                             # INTEGER ou None
                        det.get("bedrooms"),
                        det.get("bathrooms"),
                        json.dumps(det.get("features", {})),
                        json.dumps(det.get("poi", {})),
                        description,
                        basic["url"],
                        None, None, 0,                     # images remplies après
                        det.get("last_updated","Unknown"),
                    ))
                    conn.commit()

                    row = cur.fetchone()
                    if not row:
                        continue
                    listing_id = row[0]

                except Exception as db_err:
                    print(f"      ❌ DB: {db_err}")
                    conn.rollback()
                    continue

                # ── Téléchargement images ─────────────────────
                img_urls = det.get("img_urls", [])
                if basic.get("thumb") and basic["thumb"] not in img_urls:
                    img_urls.insert(0, basic["thumb"])

                thumb_path, folder, nb_imgs = download_images(
                    img_urls, prop_type, listing_id
                )

                # Mise à jour images
                cur.execute("""
                    UPDATE mubawab_listings
                    SET image_path=%s, images_folder=%s, images_count=%s
                    WHERE id=%s
                """, (thumb_path, folder, nb_imgs, listing_id))
                conn.commit()

                cat_saved   += 1
                total_saved += 1

                print(f"         ✅ id={listing_id} | "
                      f"rooms={rooms} | surface={surface}m² | "
                      f"imgs={nb_imgs} | city={loc_dict.get('city')}")

            time.sleep(random.uniform(3, 6))

        category_stats[cat_key] = cat_saved
        print(f"\n  📊 {cat_key} → {cat_saved} annonces enregistrées")

except KeyboardInterrupt:
    print("\n⚠️ Interrompu manuellement (données déjà commitées).")

finally:
    driver.quit()
    cur.close()
    conn.close()

    print(f"\n{'═'*60}")
    print(f"  🎉 TERMINÉ — {total_saved} annonces au total")
    print(f"{'═'*60}")
    for k, v in category_stats.items():
        print(f"  {k:<35} : {v:>4}")
    print(f"\n  📁 Images → {MEDIA_ROOT}/")

🚀 Démarrage — Objectif : 3000 annonces

   Structure images : media/{property_type}/{id}/image_001.jpg


════════════════════════════════════════════════════════════
  📦  APARTMENT  |  SALE
════════════════════════════════════════════════════════════

  📄 Page 1 → https://www.mubawab.tn/fr/sc/appartements-a-vendre
      → 34 annonces [sel: 'div.listingBox']
      ⬇️  [1/300] S2 en vente...
         ✅ id=1 | rooms=2 | surface=Nonem² | imgs=17 | city=Tunisie
      ⬇️  [2/300] S2 en vente...
         ✅ id=2 | rooms=2 | surface=131.39m² | imgs=22 | city=Tunisie
      ⬇️  [3/300] S3 en vente...
         ✅ id=3 | rooms=3 | surface=Nonem² | imgs=19 | city=Tunisie
      ⬇️  [4/300] Fabuleux appartement S2 à vendre...
         ✅ id=4 | rooms=2 | surface=131.0m² | imgs=13 | city=Tunisie
      ⬇️  [5/300] Appartement à vendre à Les Jardins d'El Menzah 2...
         ✅ id=5 | rooms=None | surface=67.0m² | imgs=8 | city=Tunisie
      ⬇️  [6/300] Appartement S1 RDC à vendre, Résidence MALAGA 2 ...


In [ ]:
import pandas as pd

conn = psycopg2.connect(**DB_CONFIG)

print("═"*60)
print("  📋 APERÇU DES DONNÉES")
print("═"*60)

df = pd.read_sql("""
    SELECT
        id,
        title,
        transaction_type   AS type,
        property_type      AS bien,
        rooms,
        surface,
        price,
        municipality,
        location_details->>'zone'  AS zone,
        images_count,
        scraped_at::date   AS date
    FROM mubawab_listings
    ORDER BY scraped_at DESC
    LIMIT 15
""", conn)
display(df)

print("\n" + "═"*60)
print("  📊 STATISTIQUES PAR CATÉGORIE")
print("═"*60)

stats = pd.read_sql("""
    SELECT
        property_type                           AS bien,
        transaction_type                        AS type,
        COUNT(*)                                AS total,
        COUNT(rooms)                            AS avec_rooms,
        COUNT(surface)                          AS avec_surface,
        ROUND(AVG(rooms)::numeric, 1)           AS rooms_moy,
        ROUND(AVG(surface)::numeric, 1)         AS surface_moy,
        SUM(images_count)                       AS total_images
    FROM mubawab_listings
    GROUP BY property_type, transaction_type
    ORDER BY bien, type
""", conn)
display(stats)

print("\n" + "═"*60)
print("  🧪 TEST EXTRACTION INTELLIGENTE (20 exemples)")
print("═"*60)

sample = pd.read_sql("""
    SELECT title, rooms, surface
    FROM mubawab_listings
    WHERE rooms IS NOT NULL OR surface IS NOT NULL
    LIMIT 20
""", conn)
display(sample)

conn.close()